In [ ]:
from flask import Flask , request, render_template , make_response ,jsonify ## flask 라이브러리에서 Flask import
import numpy as np
import os
import time
from urllib.request import urlretrieve 
import pandas as pd
import numpy as np
import requests as req
import json
from flask_socketio import SocketIO
from flask_cors import CORS
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

app = Flask(__name__)

CORS(app)
@app.route('/arrest_rate', methods=['POST'])
def arrest_rate():

    # 전처리
    crime = pd.read_csv('data/seoul_crime.csv', skiprows=2)
    crime.columns = ['행정구별', '총범죄발생', '총범죄검거', '살인발생', '살인검거', '강도발생', '강도검거', '강간강제추행발생', '강간강제추행검거', '절도발생', '절도검거', '폭력발생', '폭력검거']
    crime['살인검거율'] = crime['살인검거'] / crime['살인발생'] * 100
    crime['강도검거율'] = crime['강도검거'] / crime['강도발생'] * 100
    crime['강간강제추행검거율'] = crime['강간강제추행검거'] / crime['강간강제추행발생'] * 100
    crime['절도검거율'] = crime['절도검거'] / crime['절도발생'] * 100
    crime['폭력검거율'] = crime['폭력검거'] / crime['폭력발생'] * 100
    crime_arr = crime[['행정구별', '강도검거', '강간강제추행검거', '살인검거', '절도검거', '폭력검거']]
    crime_per = crime[['행정구별', '강도검거율', '강간강제추행검거율', '살인검거율', '절도검거율', '폭력검거율']]
    
    # 서울시 5대 범죄 검거 건수
    top_group_sizes = crime_arr.iloc[0, 1:].values    
    
    # 서울시 5대 범죄 검거율
    subgroup_sizes = crime_per.iloc[0, 1:].values

    i_list=[]
    
    for i in subgroup_sizes:
        i_list.append(int(i))
        
    j_list=[]
    for j in top_group_sizes:
        j_list.append(int(j))
        

    if request.method == 'POST':
        data = {
            'top_group_sizes': j_list,
            'subgroup_sizes': i_list
        }
        print(data)
        response = make_response(jsonify(data))
        print(jsonify(data))
        response.headers.add("Access-Control-Allow-Origin", "*")
        return response
    
@app.route('/crime_arr', methods=['POST'])
def crime_arr():
    
    # 전처리
    crime = pd.read_csv('data/seoul_crime.csv', skiprows=2)
    crime.columns= ['행정구별', '총범죄건수', '총범죄검거', '살인건수', '살인검거', '강도건수', '강도검거','강간강제추행건수', '강간강제추행검거', '절도건수', '절도검거', '폭력건수', '폭력검거']
    crime_occ = crime[['행정구별','살인건수', '강도건수','강간강제추행건수', '절도건수', '폭력건수']]
    categories = ['살인건수', '강도건수', '강간강제추행건수', '절도건수', '폭력건수']
    
    values = crime_occ.iloc[0, 1:].values
    
    # 서울시 5대 범죄 발생 건수
    i_list=[]
    
    for i in values:
        i_list.append(int(i))

    if request.method == 'POST':
        data = {
            'categories': categories,
            'values' : i_list
        }
        response = make_response(jsonify(data))
        response.headers.add("Access-Control-Allow-Origin", "*")
        return response

@app.route('/info', methods=['POST'])
def info():
    
    # 전처리
    na_pop = pd.read_csv('data/na_pop.csv',encoding='cp949',skiprows=2)
    na_pop.columns = ['전국','총인구수','남자인구수','여자인구수']    
    na_crime = pd.read_csv('data/na_crime.csv')    
    na_pop_cri = pd.concat([na_pop,na_crime],axis=1)    
    na_pop_cri['천명당건수']=na_pop_cri['발생건수']/(na_pop_cri['총인구수']/1000)    
    police = pd.read_csv('data/police.csv')    
    result = police.groupby('시도청')['경찰관'].sum()   
    order_of_locations = ['서울','부산','대구','인천','광주','대전','울산','세종','경기','강원','충북','충남','전북','전남','경북','경남','제주']    
    result = result.reindex(order_of_locations)    
    result = result.reset_index()    
    all_crime = pd.concat([na_pop_cri,result],axis=1)    
    all_crime['경찰관한명당범죄건수']=all_crime['발생건수']/all_crime['경찰관']
    
    crime = all_crime.iloc[:, 5].values
    pol_cri = all_crime.iloc[:, 10].values
    info_cri = all_crime.iloc[:, 7].values
    
    # 전국 시도별 범죄 발생 건수
    a_list=[]
    
    # 전국 시도별 경찰 한명당 범죄 발생 건수
    b_list=[]
    
    # 전국 시도별 천명당 범죄 발생건수
    c_list=[]


    for a in crime:
        a_list.append(int(a))
    for b in pol_cri:
        b_list.append(int(b))
    for c in info_cri:
        c_list.append(int(c))
        
    if request.method == 'POST':

        data = {
            'crime' : a_list,
            'pol_cri': b_list,
            'info_cri' : c_list
            
        }
        response = make_response(jsonify(data))
        response.headers.add("Access-Control-Allow-Origin", "*")
        return response    
    
    
if __name__ == "__main__":
    app.run(port=5003)
app = Flask(__name__)

CORS(app)
@app.route('/pre', methods=['POST'])
def pre():
    
    # 서울시 5대 범죄 데이터
    seoul_crime = pd.read_csv('data/seoul_crime.csv', skiprows=3)
    seoul_crime.columns = ['행정구','총발생','총검거','살인발생','살인검거','강도발생','강도검거','강간강제추행발생', '강간강제추행검거','절도발생','절도검거','폭력발생','폭력검거']

    # 서울시 인구수 데이터
    seo_pop = pd.read_csv('data/seo_pop.csv', skiprows=3)
    seo_pop.columns = ['합계','행정구','행정동','전체인구수','남자인구수','여자인구수','한국인구수','한국남자','한국여자','외국인구수','외국남자','외국여자']

    # 서울시 각구별 경찰서
    seoul_police_office = pd.read_csv('data/seoul_polic.csv')

    # 서울시 cctv
    seoul_cctv = pd.read_csv('data/seoul_cctv.csv', encoding='cp949')
    
    seo_gu = seoul_crime['행정구'].unique()
        
    # 각 구별 인구수
    pop_list = []
    for gu in seo_gu:
        pop_gu = seo_pop[seo_pop['행정구']==gu]['전체인구수'].sum()/2
        pop_list.append(pop_gu)
    seoul_crime['구별인구수'] = pd.Series(pop_list)
    
    # 각 구별 경찰서 수
    pof_list=[]
    for gu in seo_gu:
        pof_gu = seoul_police_office[seoul_police_office['행정구']==gu]['경찰서'].count()
        pof_list.append(pof_gu)
    seoul_crime['구별경찰서수'] = pd.Series(pof_list)
    
    # 각 구별 cctv 수
    cctv_list = []
    for gu in seo_gu:
        cctv_gu = seoul_cctv[seoul_cctv['자치구']==gu]['CCTV 수량'].sum()
        cctv_list.append(cctv_gu)
    seoul_crime['구별cctv수'] = pd.Series(cctv_list)
    
    # 임의로 만든 안전지수
    seoul_crime['안전지수'] = (
        0.4 * seoul_crime['총검거'] / seoul_crime['총발생'] +
        0.3 * seoul_crime['구별인구수'] / seoul_crime['총발생'] +
        0.2 * seoul_crime['구별경찰서수'] / seoul_crime['구별인구수'] +
        0.1 * seoul_crime['구별cctv수'] / seoul_crime['구별인구수'])
    
    features = seoul_crime[['총발생', '총검거', '구별인구수', '구별경찰서수', '구별cctv수']]
    target = seoul_crime['안전지수']
    
    # 표준화
    scaler = StandardScaler()   
    features_scaled = scaler.fit_transform(features)
    
    X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.3, random_state=42)
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_test)
    
    all_predictions = model.predict(features_scaled)
    
    mse_test = mean_squared_error(y_test, predictions)
    print(f'평균제곱오차 (test): {mse_test}')
    
    mse_all = mean_squared_error(target, all_predictions)
    print(f'평균제곱오차 (all): {mse_all}')
    
    result_df_all = pd.DataFrame({'실제값': target, '예측값': all_predictions})
    
    i_list=[]
    
    for i in  all_predictions:
        i_list.append(float(i))        

    if request.method == 'POST':
        data = {"seo_gu" : seo_gu.tolist(),
                "all_predictions": i_list}
        print(data)
        response = make_response(jsonify(data))
        print(jsonify(data))
        response.headers.add("Access-Control-Allow-Origin", "*")
        return response

if __name__ == "__main__":
    app.run(port=5003)